# Signal amplification

Insert the external trigger signal to the back of the Agilent 33220A function generator and connect its output with the external trigger input of the oscilloscope. Let the oscilloscope trigger on falling edge and set the time scale to 20 us. The function generator should be configured to output pulse signals with period of 30 ms.

The output of the horizontal DDS is connected to the horizontal amplifier. The DDS driver is configured with inverse sinc filter turned off in the subsequent experiments.

The output of the horizontal amplifier is connected to the following attentuators to not destory the oscillscope:

1. 1 dB
2. 3 dB
3. 3 dB
4. 6 dB
5. 10 dB
6. 10 dB

With the damping power increasing towards the oscilloscope such that the heat is distributed more evenly.

In [4]:
import os
import time
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

datadir = '../../dataset/signal/amplification'

os.makedirs(datadir, exist_ok=True)

In [2]:
import control

scope = control.MSOX6004A('172.22.22.30')
fngen = control.FG33250A2('172.22.22.36')

scope.holla(), fngen.holla()

('AGILENT TECHNOLOGIES,MSO-X 4054A,MY57310545,07.20.2017102615\n',
 'Agilent Technologies,33220A,MY43003480,1.05-1.05-22-1\n')

In [3]:
duration = 30e-3

delays = np.linspace(0, duration, 300)

Insert the external trigger signal to the back of the Agilent 33220A function generator and connect its output with the external trigger input of the oscilloscope. Let the oscilloscope trigger on falling edge and set the time scale to 20 us. The function generator should be configured to output pulse signals with period of 30 ms. Now connect the channel 1 input of the oscilloscope with the output of the horizontal DDS. Check that the DDS driver server is configured to have the sinc filter turned off.

## Digital Ramp

Frequency response with digital ramp.

### Horizontal DDS

In [8]:
control.aod_h(frequency=[80e6, 120e6], duration=duration)

data = []

for t in delays:
    fngen.width(t)
    
    data.append((t, scope.capture()))

for (t, df) in data:
    df.to_hdf(os.path.join(datadir, 'horizontal.h5'), f'{t:.4}', format='table', complevel=9)

### Vertical DDS

Now connect the vertical DDS with the oscillscope output. Check again that the DDS driver has the inverse sinc filter turned off.

In [7]:
control.aod_v(frequency=[80e6, 120e6], duration=duration)

data = []

for t in delays:
    fngen.width(t)
    
    data.append((t, scope.capture()))

for (t, df) in data:
    df.to_hdf(os.path.join(datadir, 'vertical.h5'), f'{t:.4}', format='table', complevel=9)

## Sampling

Frequency sweep through sampling.

### Vertical DDS

In [6]:
data = []

for f in np.linspace(80e6, 120e6, 200):
    control.aod_v(frequency=f)
    
    time.sleep(.5)

    U = scope.data(1).voltage.max()
    
    data.append((f, U))
    
df = pd.DataFrame(data, columns=['frequency', 'amplitude'])
df.to_hdf(os.path.join(datadir, 'sampled.h5'), 'vertical', complevel=9)

### Horizontal DDS

In [5]:
data = []

for f in np.linspace(80e6, 120e6, 200):
    control.aod_h(frequency=f)

    time.sleep(.5)
    
    U = scope.data().voltage.max()
    
    data.append((f, U))
    
df = pd.DataFrame(data, columns=['frequency', 'amplitude'])
df.to_hdf(os.path.join(datadir, 'sampled.h5'), 'horizontal', complevel=9)

In [43]:
df